# Setup stuff: Connectivity

In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


In [ ]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


#  Actual analytics ..

In [ ]:

from katana.remote import analytics


In [ ]:

#  Getting the airport node id
#

#  l_airport = "MKE"
#  l_airport = "ORD"
#  l_airport = "DEN"
l_airport = "SJC"


l_query  = """
   MATCH (n: Airport) 
   WHERE n.airport_code = '{0}'
   RETURN ID(n) AS my_id
   """.format(l_airport)

l_airport_id = my_graph.query_unpaginated(l_query)["my_id"][0]

print("Airport id:  %s   %d" % (l_airport, l_airport_id))
print("")


In [ ]:

#  Running analytics.sssp() ..

#  From above
#
l_source_nodeid       = str(l_airport_id)


#  Below:  this currently fails on DISTANCE|NUM_HOPS.  Works with None.
#

#  Edge property
#
l_weight_property     = "DISTANCE"                 #  Edge property
#  l_weight_property     = "NUM_HOPS"
#  l_weight_property     = None   

l_max_iterations      = 10000 


l_result = analytics.sssp(
   my_graph,
      #
   source               = l_source_nodeid,
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   )

print(l_result)


#  Error,
#
#     ---------------------------------------------------------------------------
#     CanceledError                             Traceback (most recent call last)
#     Cell In[8], line 20
#          15 #   l_weight_property     = None   
#          17 l_max_iterations      = 10000 
#     ---> 20 l_result = analytics.sssp(
#          21    my_graph,
#          22       #
#          23    source               = l_source_nodeid,
#          24    weight_property      = l_weight_property,
#          25    max_iterations       = l_max_iterations,
#          26    )
#          28 print(l_result)
#          31 #  Error,
#          32 #
#          33 #     ---------------------------------------------------------------------------
#        (...)
#          75 #     
#          76 #     CancelledError: 
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:168, in async_to_sync.<locals>.wrapper(timeout, *args, **kwargs)
#         165 if convert_objects:
#         166     registry = AsyncToSyncClassRegistry.get()
#         167     return registry.async_to_sync(
#     --> 168         wait_for(
#         169             async_func(
#         170                 *(registry.sync_to_async(a) for a in args),
#         171                 **{k: registry.sync_to_async(v) for k, v in kwargs.items()},
#         172             ),
#         173             timeout=timeout,
#         174         )
#         175     )
#         176 return wait_for(async_func(*args, **kwargs), timeout=timeout)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:147, in wait_for(coro, timeout)
#         145 try:
#         146     future = asyncio.run_coroutine_threadsafe(timeout_coro, loop=AsyncRunnerThread.get().loop)
#     --> 147     return future.result()
#         148 except KeyboardInterrupt:
#         149     inner_future.cancel()
#     
#     File /opt/conda/lib/python3.8/concurrent/futures/_base.py:444, in Future.result(self, timeout)
#         442     raise CancelledError()
#         443 elif self._state == FINISHED:
#     --> 444     return self.__get_result()
#         445 else:
#         446     raise TimeoutError()
#     
#     File /opt/conda/lib/python3.8/concurrent/futures/_base.py:389, in Future.__get_result(self)
#         387 if self._exception:
#         388     try:
#     --> 389         raise self._exception
#         390     finally:
#         391         # Break a reference cycle with the exception in self._exception
#         392         self = None
#     
#     File /opt/conda/lib/python3.8/asyncio/tasks.py:455, in wait_for(fut, timeout, loop)
#         450     warnings.warn("The loop argument is deprecated since Python 3.8, "
#         451                   "and scheduled for removal in Python 3.10.",
#         452                   DeprecationWarning, stacklevel=2)
#         454 if timeout is None:
#     --> 455     return await fut
#         457 if timeout <= 0:
#         458     fut = ensure_future(fut, loop=loop)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/analytics/__init__.py:143, in sssp(graph, source, weight_property, delta, max_iterations, use_gpu)
#         130 else:
#         131     data = {
#         132         "analytics": {
#         133             "sssp": {
#        (...)
#         140         },
#         141     }
#     --> 143 return await graph._run_asynchronously_and_read_from_parquet(data)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/graph.py:306, in Graph._run_asynchronously_and_read_from_parquet(self, data)
#         303 async def _run_asynchronously_and_read_from_parquet(self, data) -> DataFrame:
#         304     data = self._add_operation_metadata(data)
#     --> 306     result = await self._client._run_on_graph_and_wait(self, data)
#         308     files = result["result_files"]
#         309     operation_id = result["operation_id"]
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/client.py:333, in Database._run_on_graph_and_wait(self, graph, data)
#         331 while attempt < max_attempts:
#         332     try:
#     --> 333         return await self._wait_op(op)
#         334     except errors.ServerCommunicationError:
#         335         logging.warning(f"error while waiting, retrying (attempt={attempt})")
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/client.py:270, in Database._wait_op(self, op)
#         268 async with _progress_bar() as pbar:
#         269     async with wait_fn(operation_id) as stream:
#     --> 270         async for update in stream:
#         271             status = update["status"]
#         273             for entry in update.get("progress", []):
#     
#     File /opt/conda/lib/python3.8/site-packages/asyncstdlib/builtins.py:445, in map(function, *iterable)
#         443 async for args in args_iter:
#         444     result = function(*args)
#     --> 445     yield await result
#     
#     File /opt/conda/lib/python3.8/site-packages/asyncstdlib/_core.py:134, in force_async.<locals>.async_wrapped(*args, **kwargs)
#         133 async def async_wrapped(*args: Any, **kwargs: Any) -> T:
#     --> 134     return call(*args, **kwargs)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/rpc/client.py:171, in _OperationClient._event_stream.<locals>.parse_stream(line)
#         169     raise errors.OperationError(operation_id + " " + data["status_message"])
#         170 if status == "Canceled":
#     --> 171     raise errors.CanceledError()
#         173 return data
#     
#     CanceledError: 
        
